The following packages are installed in the snowflake python notebook
- matplotlib
- nbformat
- plotly
- python-kaleido
- statsmodels
- python - 3.10.*
- streamlit 1.39.1


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go

pio.renderers.default = "png"

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# load in the unfiltered Charity table
sql = """
SELECT
    *
FROM GIVE.STAGING.CHARITY
"""

df = session.sql(sql).to_pandas()

In [ ]:
# plot charity distribution by registration status
count_df = (
    df["CHARITY_REGISTRATION_STATUS"]
    .value_counts()
    .reset_index()
)

count_df.columns = ["CHARITY_REGISTRATION_STATUS", "count"]

fig = px.pie(
    count_df,
    names="CHARITY_REGISTRATION_STATUS",
    values="count",
    title="Distribution of Charities by Registration Status"
)

fig.update_traces(
    texttemplate="%{label}<br>%{value:,} (%{percent})",
    textposition="inside"
)

fig.update_layout(
    uniformtext_minsize=10,
    uniformtext_mode="hide"
)

fig

In [ ]:
%sql
-- look at schema of filtered table
describe table give.dim.charity;

In [ ]:
# load in filtered charity table
sql = """
SELECT
    *
FROM GIVE.DIM.CHARITY
"""

df = session.sql(sql).to_pandas()

In [ ]:
df.info()

### Get Summary Stats of Numeric Variables

In [ ]:
df.describe()

### Plot distribution of Latest Income

In [ ]:
# plot log distrubtion of latest income
column_name = "LATEST_INCOME"

def generate_box_plot(df : pd.DataFrame, column_name : str):

    fig = px.box(
        df,
        y=column_name,
        log_y=True,
        title="Box plot of Latest Income (log scale)",
    )
    
    fig.update_traces(marker_color="blue")
    
    return fig

generate_box_plot(df, 'LATEST_INCOME')

### Correlation between Age and Latest Income

In [ ]:
# Clean data
x_col = "AGE"
y_col = "LATEST_INCOME"
df_xy = df[[x_col, y_col]].dropna()

fig = px.scatter(
    df_xy,
    x=x_col,
    y=y_col,
    trendline="ols",   # ordinary least squares
    title=f"Correlation between {x_col} and {y_col}",
    labels={
        "X_COL": x_col,
        "Y_COL": y_col,
    },
    log_y=True,
    # log_x=True,
)

# fig.update_layout(template="plotly_white")
fig

### Correlation between Latest Expenditure and Income

In [ ]:
# Clean data
x_col = "LATEST_INCOME"
y_col = "LATEST_EXPENDITURE"
df_xy = df[[x_col, y_col]].dropna()

fig = px.scatter(
    df_xy,
    x=x_col,
    y=y_col,
    trendline="ols",   # ordinary least squares
    title=f"Correlation between {x_col} and {y_col}",
    labels={
        "X_COL": x_col,
        "Y_COL": y_col,
    },
    log_x=True,
    log_y=True,
)

fig.update_layout(template="plotly_white")
fig

### Define helper functions for plotting graphs

In [ ]:
# Define pie-plot function

def generate_pie_plot(df : pd.DataFrame, column_name : str, distributed_by : str):
    count_df = (
        df[column_name]
        .value_counts()
        .reset_index()
    )

    count_df.columns = [column_name, "count"]

    fig = px.pie(
        count_df,
        names=column_name,
        values="count",
        title=f"Distribution of Charities by {distributed_by}"
    )
    
    fig.update_traces(
        texttemplate="%{label}<br>%{value:,} (%{percent})",
        textposition="inside"
    )
    
    fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode="hide"
    )
    
    return fig


# Define bar-plot function
def generate_barplot(df : pd.DataFrame, x_column_name :str, y_column_name : str, distributed_by :str, agg_method = "mean"):
    agg_df = (
        df.groupby(x_column_name, as_index=False)
          .agg(avg_value=(y_column_name, agg_method))
    )
    
    fig = px.bar(
        agg_df,
        x=x_column_name,
        y="avg_value",
        title=f"{agg_method.capitalize()} {y_column_name.replace('_', ' ').title()} by {distributed_by}",
        labels={
            "GROUP_COL": x_column_name,
            "avg_value": f"{agg_method.title()} {y_column_name.replace('_', ' ').title()}",
        },
    )
    
    fig.update_traces(texttemplate="%{y:,.0f}", textposition="inside")
    
    return fig


### Distribution and Impact by Charity Type

In [ ]:
# CHARITY_TYPE distribution
generate_pie_plot(df, "CHARITY_TYPE", "Charity Type")

In [ ]:
# plot mean latest income by charity type
generate_barplot(df, "CHARITY_TYPE", "LATEST_INCOME", "Charity Type", "mean")

In [ ]:
# plot mean latest income by charity type
generate_barplot(df, "CHARITY_TYPE", "LATEST_INCOME", "Charity Type", "median")

### Impact of Gift Aid

In [ ]:
generate_pie_plot(df, "IS_GIFT_AID", "Being Gift Aid Registered")

In [ ]:
# plot mean latest income by gift aid
generate_barplot(df, "IS_GIFT_AID", "LATEST_INCOME", "Being Gift Aid Registered", "mean")

In [ ]:
# plot mean latest income by gift aid
generate_barplot(df, "IS_GIFT_AID", "LATEST_INCOME", "Being Gift Aid Registered", "median")

### Impact of Having Land

In [ ]:
generate_pie_plot(df, "HAS_LAND", "Having Land")


In [ ]:
# plot mean latest income by having land
generate_barplot(df, "HAS_LAND", "LATEST_INCOME", "Having Land", "mean")

In [ ]:

# plot mean latest income by having land
generate_barplot(df, "HAS_LAND", "LATEST_INCOME", "Having Land", "median")

### Impact of Having a Website

In [ ]:
generate_pie_plot(df, "HAS_WEBSITE", "Website Ownership")

In [ ]:
# plot mean latest income by having a website
generate_barplot(df, "HAS_WEBSITE", "LATEST_INCOME", "Website Ownership", "mean")

In [ ]:
# plot mean latest income by having a website
generate_barplot(df, "HAS_WEBSITE", "LATEST_INCOME", "Website Ownership", "median")